## Imports

In [1]:
import pandas as pd

In [35]:
df = pd.read_csv('../data/base_livros.csv')

### 1. /api/v1/books

In [36]:
df

,id,title,price,rating,availability,category,image
0,1,It's Only the Himalayas,45.17,2,In stock,Travel,https://books.toscrape.com/media/cache/27/a5/2...
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,4,In stock,Travel,https://books.toscrape.com/media/cache/57/77/5...
2,3,See America: A Celebration of Our National Par...,48.87,3,In stock,Travel,https://books.toscrape.com/media/cache/9a/7e/9...
3,4,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,In stock,Travel,https://books.toscrape.com/media/cache/d5/bf/d...
4,5,Under the Tuscan Sun,37.33,3,In stock,Travel,https://books.toscrape.com/media/cache/98/c2/9...
...,...,...,...,...,...,...,...
994,995,Why the Right Went Wrong: Conservatism--From G...,52.65,4,In stock,Politics,https://books.toscrape.com/media/cache/db/1b/d...
995,996,Equal Is Unfair: America's Misguided Fight Aga...,56.86,1,In stock,Politics,https://books.toscrape.com/media/cache/00/11/0...
996,997,Amid the Chaos,36.58,1,In stock,Cultural,https://books.toscrape.com/media/cache/52/46/5...
997,998,Dark Notes,19.19,5,In stock,Erotica,https://books.toscrape.com/media/cache/6e/4e/6...


In [41]:
dict_books = {}
for i in range(1, df.shape[0]+1):
    dict_books[i] = df[df['id'] == i]['title'][i-1]

# dict_books

In [43]:
# df.set_index('id')['title'].to_dict()

### 2. /api/v1/categories

In [49]:
# {'categories':list(df['category'].unique())}

### 3. /api/v1/books/{id}

In [58]:
id = 7

df[df['id'] == id].to_dict(orient = 'records')

[{'id': 7,
  'title': 'The Great Railway Bazaar',
  'price': 30.54,
  'rating': 1,
  'availability': 'In stock',
  'category': 'Travel',
  'image': 'https://books.toscrape.com/media/cache/76/de/76de41867f323d7f1f4fbe2fdfc1b2ba.jpg'}]

### 4. /api/v1/books/search

In [65]:
title = 'murder'
category = 'mystery'

df_query = df.copy()

if title:
    df_query = df_query[df_query['title'].str.lower().str.contains(title.lower())]

if category:
    df_query = df_query[df_query['category'].str.lower().str.contains(category.lower())]

df_query.to_dict(orient = 'records')

[{'id': 15,
  'title': 'A Murder in Time',
  'price': 16.64,
  'rating': 1,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/11/aa/11aaad48b5f15e262456ca65294084da.jpg'},
 {'id': 16,
  'title': 'The Murder of Roger Ackroyd (Hercule Poirot #4)',
  'price': 44.1,
  'rating': 4,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/29/fe/29fe70b1b2e5a9ba61d4bd331255e19e.jpg'},
 {'id': 23,
  'title': 'Murder at the 42nd Street Library (Raymond Ambler #1)',
  'price': 54.36,
  'rating': 4,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/c3/8d/c38d65cd155b67ca025f0655bd1bb095.jpg'},
 {'id': 30,
  'title': "The Bachelor Girl's Guide to Murder (Herringford and Watts Mysteries #1)",
  'price': 52.3,
  'rating': 5,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/95/d7/95d754

### 5. /api/v1/stats/overview

In [72]:
dict_results = {
    'total_books': df.shape[0],
    'mean_price': round(df['price'].mean(), 2),
    'ratings_distribution': df['rating'].value_counts().to_dict()
}

### 6. /api/v1/stats/categories

In [100]:
# r.category: {
#             'n_books': r.n_books,
#             'min_price': float(r.min_price),
#             'max_price': float(r.max_price),
#             'mean_price': round(float(r.mean_price), 2),
#             'mean_rating': round(float(r.mean_rating), 2)
#         }

stats_categories = df.groupby('category').agg({'title':'count', 'price':['min', 'max', 'mean'], 'rating':'mean'})
stats_categories = stats_categories.round(2).reset_index()

stats_categories.columns = ['_'.join(map(str, col)).strip('_') for col in stats_categories.columns.values]

lista_stats_cats = {}
for category in stats_categories['category'].unique():
    lista_stats_cats[category] = stats_categories[stats_categories['category'] == category].drop(columns = 'category').to_dict(orient = 'records')[0]

# lista_stats_cats

In [104]:
lista_stats_cats = (
    df.groupby('category')
    .agg(
        n_books = ('title', 'count'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        rating_mean = ('rating', 'mean')
    )
    .round(2)
    .to_dict(orient='index')
)

# lista_stats_cats

### 7. /api/v1/books/top-rated

In [106]:
df[df['rating'] == df['rating'].max()].to_dict(orient = 'records')

[{'id': 11,
  'title': '1,000 Places to See Before You Die',
  'price': 26.08,
  'rating': 5,
  'availability': 'In stock',
  'category': 'Travel',
  'image': 'https://books.toscrape.com/media/cache/d7/0f/d70f7edd92705c45a82118c3ff6c299d.jpg'},
 {'id': 20,
  'title': 'A Time of Torment (Charlie Parker #14)',
  'price': 48.35,
  'rating': 5,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/e8/c0/e8c0ba15066bab950ae161fd60949b9a.jpg'},
 {'id': 29,
  'title': 'What Happened on Beale Street (Secrets of the South Mysteries #2)',
  'price': 25.37,
  'rating': 5,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/c7/ab/c7abb5e32bd37118a87523dcee0a70a6.jpg'},
 {'id': 30,
  'title': "The Bachelor Girl's Guide to Murder (Herringford and Watts Mysteries #1)",
  'price': 52.3,
  'rating': 5,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/med

### 8. /api/v1/books/price-range?min={min}&max={max}

In [112]:
float('a')

ValueError: could not convert string to float: 'a'

In [118]:
def is_float(valor: str) -> bool:
    try:
        float(valor.replace(',', '.'))
        return True
    except (ValueError, TypeError):
        return False

In [121]:
is_float('25,6')

True

In [110]:
min_price = 10
max_price = 20

df_query = df.copy()

if min_price:
    df_query = df_query[df_query['price'] >= min_price]

if max_price:
    df_query = df_query[df_query['price'] <= max_price]

df_query.to_dict(orient = 'records')

[{'id': 13,
  'title': 'In a Dark, Dark Wood',
  'price': 19.63,
  'rating': 1,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/23/85/238570a1c284e730dbc737a7e631ae2b.jpg'},
 {'id': 15,
  'title': 'A Murder in Time',
  'price': 16.64,
  'rating': 1,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/11/aa/11aaad48b5f15e262456ca65294084da.jpg'},
 {'id': 18,
  'title': 'That Darkness (Gardiner and Renner #1)',
  'price': 13.92,
  'rating': 1,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/44/9e/449ed681142bc336646abee754e96639.jpg'},
 {'id': 19,
  'title': 'Tastes Like Fear (DI Marnie Rome #3)',
  'price': 10.69,
  'rating': 1,
  'availability': 'In stock',
  'category': 'Mystery',
  'image': 'https://books.toscrape.com/media/cache/3c/91/3c91d97266bd6dda322089695fb46daf.jpg'},
 {'id': 21,
  'title': 'A Study in Scarlet